In [12]:
import json
from collections import defaultdict
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

In [ ]:
file = open("dicionario_final.json", encoding = "utf-8")
dici_final = json.load(file)
file.close()

In [14]:
# modelo BioBERTpt
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
# Extrair termos e definições 
# Campos a usar: "Descrição" e "termo_popular"

termos = []
definicoes = []

for termo, info in dici_final.items():
    # 1) Obter termo_popular sempre como lista
    termo_popular = info.get("termo_popular", [])
    if isinstance(termo_popular, str):
        termo_popular = [termo_popular]

    # 2) Obter campo "Descrição" (pode ser string ou lista)
    desc = info.get("Descrição") or info.get("descricao")
    if isinstance(desc, list):
        defin = " ".join([d for d in desc if d])  # junta todos removendo strings vazias
    else:
        defin = desc

    # 3) Se não houver "Descrição", usar todo termo_popular
    if not defin and termo_popular:
        defin = " ".join(termo_popular)

    # 4) Se temos algo em defin, guardar
    if defin:
        termos.append(termo)
        definicoes.append(defin.strip())

# Exemplo de saída
print(termos[:50])
print(definicoes[:50])


['Acalabrutinib', 'Ácido desoxirribonucleico', 'Ácido ribonucleico', 'Limitação do esforço terapêutico', 'ADG20', 'Aerossol', 'Agente biológico', 'Ageusia', 'Agrupamento', 'Isolamento', 'Isolado de vírus', 'Alta n f; alta hospitalar n f; alta médica n f', 'Alunacedase alfa', 'Ambroxol', 'AMY-101', 'Anacinra', 'Antiálgico', 'Anosmia', 'Ansiedade de antecipação', 'Antibiótico', 'Anticorpo', 'Antigénio', 'Antipirético', 'Antirretroviral ARV', 'Antiviral', 'Antivírico', 'Apilimod', 'Achatar a curva', 'Apremilast', 'Arritmia', 'ASC09F', 'Assintomático', 'Ensaio clínico aleatorizado', 'Ensaio clínico em ocultaçao', 'Ensaio clínico em dupla ocultação', 'Ensaio clínico em tripla ocultação', 'Ensaio clínico controlado', 'Ensaio de intervenção comunitária', 'Astenia', 'Tratamento da dor', 'Cuidados ao domicílio', 'Cuidados hospitalares', 'Cuidados primários', 'Atibuclimab', 'ATYR1923', 'Auto-inflamação', 'Auto-amostragem', 'Autoridade de saúde', 'Aviptadil', 'Azitromicina']
['Fàrmac antineoplàst

In [16]:
# Função para obter embedding da frase (média dos tokens)
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    # Média dos embeddings dos tokens (dimensão 1)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [17]:
# Gerar embeddings para todas as definições
embeddings = []
for definicao in tqdm(definicoes, desc="Calculando embeddings"):
    embeddings.append(get_embedding(definicao))
embeddings = np.stack(embeddings)

Calculando embeddings:   0%|          | 0/3093 [00:00<?, ?it/s]

Calculando embeddings: 100%|██████████| 3093/3093 [03:46<00:00, 13.66it/s]


In [22]:
# Calcular similaridade de cosseno
sim_matrix = cosine_similarity(embeddings)

In [24]:
# Para cada termo, selecionar os 5 mais similares (excluindo o próprio termo)
top_similares = {}
for idx, termo in enumerate(termos):
    similares_idx = np.argsort(sim_matrix[idx])[::-1][1:6]
    similares = [termos[i] for i in similares_idx]
    top_similares[termo] = similares

In [25]:
# Exemplo de resultado
for termo, similares in list(top_similares.items())[-5:]:
    print(f"{termo}: {similares}")

Teste pcr para covid-19: ['Teste serológico para covid-19', 'Teste de carga viral', 'Elisa indireto', 'Biópsia', 'Ubiquitina']
Teste serológico para covid-19: ['Teste pcr para covid-19', 'Elisa indireto', 'Adjuvante genético', 'Imunossensor amperométrico', 'Teste de carga viral']
Transmissão na comunidade: ['Sintomatologia', 'Metástase', 'Toxoplasmose', 'Quarentena ou isolamento profilático', 'R']
Transmissão direta: ['Transmissão indireta', 'Etiqueta respiratória', 'Quartos de pressão negativa', 'Distanciamento social', 'Vírus respiratório sincicial']
Transmissão indireta: ['Transmissão direta', 'Etiqueta respiratória', 'Distanciamento social', 'Quartos de pressão negativa', 'Dermatomiosite']


In [ ]:
# Acrescentar a lista de similares ao dicionário original
for termo, similares in top_similares.items():
    if termo in dici_final:
        dici_final[termo]["Similares"] = similares

with open("dicionario_final_similares.json", "w", encoding="utf-8") as f:
    json.dump(dici_final, f, ensure_ascii=False, indent=4)